# DSC 80: Lab 03

### Due Date: Monday Oct 21, Noon (12:00 PM)

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding work will be developed in an accompanying `lab**.py` file, that will be imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying python file will be tested (a la DSC 20),
2. The notebook will be graded (for graphs and free response questions).

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for working in the Notebook**:
- The notebooks serve to present you the questions and give you a place to present your results for later review.
- The notebook on *lab assignments* are not graded (only the `.py` file).
- Notebooks for PAs will serve as a final report for the assignment, and contain conclusions and answers to open ended questions that are graded.
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `.py` file.

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the lab! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `lab**.py` (much like we do in the notebook).
- Always document your code!

### Importing code from `lab**.py`

* We import our `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab**.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab**.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab**.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab**` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import lab03 as lab

In [4]:
import os
import pandas as pd
import numpy as np

---

# Hypothetically speaking...

In this section we'll develop an intuition for the terms and structure of hypothesis testing -- it's nothing to be afraid of!

The first step is always to define what you're looking at, create your hypotheses, and set a level of significance.  Once you've done that, you can find a p-value which is related to your test statistic.

If all of these words are scary: look at the lecture notebook, the textbook references, and don't forget to think about the real-world meaning of the terms!  The following example describes a real-world scenario, so you can think of it in a normal lens.

**Question 1: Faulty tires**

A tire manufacturer tests whether a set of tires meets the company's performance standards by checking:

> In 60 out of 100 tests, if a Honda CRV can come to a complete stop from 60 mph in fewer than 108 feet.

That is, 60% of the time, the stopping distance of the car should be above average for the car (outfitted with generic tires). The factory is wondering if a current run of tires is up to standard, so they choose a random set of tires from the production line to test their performance, and bring the car to a complete stop from 60 mph a total of 100 times. Then they ask:

> Are these tires faulty? Or are they safe?


Which of the following are valid null hypotheses that address the question we are trying to answer, using the data we are given?  Which are valid alternative hypotheses?

Outfitted with that set of tires, the car:
1. has a 60 mph stopping distance under 108 feet, at least 60% of the time.
1. has a 60 mph stopping distance under 108 feet, at most 60% of the time.
1. has a 60 mph stopping distance under 108 feet, equal to 60% of the time.
1. has at least as short stopping distance to the same car with generic tires, at least 60% of the time.
1. has at least as short stopping distance to the same car with generic tires, at most 60% of the time.
1. has at least as short stopping distance to the same car with generic tires, roughly 60% of the time.
1. is as safe as the car with generic tires.
1. causes the car to stop in a shorter distance.


Write a function `car_null_hypoth` which takes zero arguments and returns a list of the valid null hypotheses.  
Write a function `car_alt_hypoth` which takes zero arguments and returns a list of the valid alternative hypotheses.

Which of the following are valid test statistics for our question?

1. The average number of feet the car took to come to a complete stop in 100 attempts.
1. The number of times the car stopped in under 108 feet in 100 attempts.
1. The number of attempts it took before the car stopped in under 108 feet.
1. The proportion of attempts the car successfully stopped in under 108 feet.

Write a function `car_test_stat` which takes zero arguments and returns a list of valid test statistics.

The p-value is calculated as how likely it is to find something as extreme or more extreme than our observed test statistic.  To do this, we assume the null hypothesis is true, and then define "extremeness" based on the alternative hypothesis.

Why don't we just look at the probability of finding our observed test statistic?

1. Because our observed test statistic isn't extreme.
2. Because the probability of finding our observed test statistic equals the probability of finding something more extreme.
3. Because the probability of finding our observed test statistic is essentially zero.
4. Because our null hypothesis isn't suggesting equality.
5. Because our alternative hypothesis isn't suggesting equality.

Write a function `car_p_value` which takes zero arguments and returns the correct reason.

# Grouping: Google Play Store

The questions below analyze a dataset of Google Play Store apps. The dataset has been preprocessed slightly for your convenience.

Columns:
* `App`: App Name
* `Category`: App Category
* `Rating`: Average App Rating
* `Reviews`: Number of Reviews
* `Size`: Size of App
* `Installs`: Binned Number of Installs
* `Type`: Paid or Free
* `Price`: Price of App
* `Content Rating`: Age group the app is targeted at
* `Last Updated`: Last Updated Date


Link: https://www.kaggle.com/lava18/google-play-store-apps

**Question 2**

First, we'd like to do some basic cleaning to this dataset to better analyze it.
In the function `clean_apps`, which takes the Play Store dataset as input, clean as follows and return the cleaned df:
* Convert `Reviews` to type int.
* Strip all letters from the ends of `Size`, convert all units to unit kilobyte, and convert the column to type float (Hint: all Sizes end in either M (megabyte) or k (kilobyte); a helper function may be useful here).
* Strip the '+' from the ends of `Installs`, remove the commas, and convert it to type int.
* Since `Type` is binary, change all the 'Free's to 1 and the 'Paid's to 0.
* Strip dollar mark in `Price` and convert it to correct numeric data type.
* Strip all but the year (e.g. 2018) from `Last Updated` and convert it to type int.

Please return a *copy* of the original dataframe; don't alter the original.

In [5]:
play_fp = os.path.join('data', 'googleplaystore.csv')
play = pd.read_csv(play_fp)

**Question 2 (Continued)**

Now, we can do some basic exploration.

In the function `store_info`, find the following using the **cleaned** dataframe:
* Find the year with the highest median `Installs`, among all years with at least 100 apps.
* Find the `Content Rating` with the highest minimum `Rating`.
* Find the `Category` has the highest average price.
* Find the `Category` with lowest average rating, among apps that have at least 1000 reviews.

and return these values in a list.

*Remark:* Note that the last question is asking you to compute the *average of averages* (the 'Rating' column contains the average rating of an app) -- such analyses are prone to occurrences of Simpson's Paradox. Considering apps with at least 1000 reviews helps limit the effect of such [ecological fallacies](https://afraenkel.github.io/practical-data-science/05/understanding-aggregations.html#reversing-aggregations-ecological-fallacies).



### Transforming Apps review count by App category

A reasonable question that we may ask after cleaning the apps dataset is that how popular each app is. One way of measuring popularity of apps is by studying its reveiw count within their respective category. 

**Question 3**
* Create a function `std_reviews_by_app_cat` that takes in a **cleaned** dataframe and outputs a dataframe with 
    - the same rows as the input,
    - two columns given by `['Category', 'Reviews']`,
    - where the `Reviews` columns are *standardized by app category* -- that is, the number of reviews for every app is put into the standard units for the category it belongs to. For a review of standard units, see the [DSC 10 Textbook](https://www.inferentialthinking.com/chapters/15/1/Correlation)
    - *Hint*: use the methoc `groupby` and `transform`.
* Lastly, create a function `su_and_spread` that returns a list of two items (hard-code your answers):
    - Consider the following scenario: half of the apps in the category 'FAMILY' receives ratings of 0 stars while the other
    half has rating of 5 stars. Similarly, the ‘MEDICAL' category has half 1-star and half 4-star apps.
    Which app would have a higher rating after standarization? The five stars in the family category or the four stars in the
    medical one. Answer with the name of the corresponding category ('FAMILY'/'MEDICAL') or use 'equal' if you think both
    rating would be the same after standarization. (Don't worry about the uppercase but do be careful with the spelling). 
    - Which category type has the biggest "spread" of review count?
    

In [39]:
cleaned = lab.clean_apps(play)

### Facebook Friends

**Question 4**

A group of students decide to send out a survey to their Facebook friends. Each student asks 1000 of their friends for their first and last name, the company they currently work at, their job title, their email, and the university they attended. Combine all the data contained in the files `survey*.csv` (within the `responses` folder within the data folder) into a single dataframe. The number of files and the number of rows in each file may vary, so don't hardcode your answers!

Create a function `read_survey` which takes in a directory path (containing files `survey*.csv`), and outputs a dataframe with six columns titled: `first name`, `last name`, `current company`, `job title`, `email`, `university` (in that order). 

*Hint*: You can list the files in a directory using `os.listdir`.

*Remark: You may have to do some cleaning to make this possible!*

Create a function `com_stats` that takes in in a dataframe and returns a list containing: the most common first name, job held, university attended, and current company (in that order) for people with emails that end in ".com". If there are ties for the most common value, give the value with the "largest size" (as defined by the python string ordering).

In [ ]:
dirname = os.path.join('data', 'responses')
lab.read_survey(dirname)

### Combining Data
**Question 5**

Every week, a professor sends out an extra credit survey asking for students' favorite things (animals, movies, etc). 
- Each student who has completed at least 75% of the surveys receives 5 points of extra credit.
- If at least 90% of the class answers at least one of the questions (ex. favorite animal), *everyone* in the class receives 1 point of extra credit. This overall class extra credit only applies once (ex. If 95% of students answer favorite color and 91% answer favorite animal, the entire class still only receives 1 extra point as a class).

Create a function `combine_surveys` which takes in a directory path (containing files `favorite*.csv`) and combines all of the survey data into one DataFrame, indexed by student ID (a value 0 - 1000).

Create a function `check_credit` which takes in a DataFrame with the combined survey data and outputs a DataFrame of the names of students and how many extra credit points they would receive, indexed by their ID (a value 0-1000)

In [ ]:
dirname = os.path.join('data', 'extra-credit-surveys')
df = lab.combine_surveys(dirname)
df.head()

### Joining pets and owners

**Question 6**

You are analyzing data from a veterinarian clinic. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, and available procedures and history. The column names are self-explanatory. These dataframes are provided to you:
-  `owners` stores the customer information, where every `OwnerID` is unique (verify yourself).
-  `pets` stores the pet information. Each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records. Each procedure is given to a pet in `pets`.

You want to answer the following questions:

1. How many pets in the dataset have had at least one procedure? Note that some pets are registered but haven't had any procedure performed. Create a function `at_least_once` that takes in `pets`, `procedure_history` and returns the number of pets as an integer.

2. What is the name of each customer's pet(s)? Create a function `pet_name_by_owner` that takes in `owners`, `pets` and returns a Series that holds the pet name (as a string) indexed by owner's (first) name. If an owner has multiple pets, the corresponding value should be a list of names as strings.

3. How much does each customer spend in total on procedures? Create a function `total_cost_per_owner` that returns a Series that contains the sum of money that an owner has spent on their pets' procedures, indexed by `OwnerID`. Hint: think of what makes a procedure unique in the context of this dataset.

In [ ]:
owners_fp = os.path.join('data', 'pets', 'Owners.csv')
pets_fp = os.path.join('data', 'pets', 'Pets.csv')
procedure_detail_fp = os.path.join('data', 'pets', 'ProceduresDetails.csv')
procedure_history_fp = os.path.join('data', 'pets', 'ProceduresHistory.csv')

In [ ]:
owners = pd.read_csv(owners_fp)
pets = pd.read_csv(pets_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)
procedure_history = pd.read_csv(procedure_history_fp)

In [ ]:
procedure_detail

In [ ]:
lab.pet_name_by_owner(owners, pets)

## Congratulations! You're done!

* Submit the lab on Gradescope